# Tiền xử lý

In [1]:
import pandas as pd

# Đọc dữ liệu từ file csv
fall_dataset = pd.read_csv("data/data.csv")

# Xóa các hàng có giá trị bị thiếu hoặc trùng lặp
fall_dataset.drop_duplicates(inplace=True)
fall_dataset.dropna(inplace=True)

In [2]:
fall_dataset.head()

,1,2,3,4,5,6,7,8,9,10,...,328,329,330,331,332,333,334,335,336,label
0,0.482556,0.301600,-0.016666,0.478341,0.489260,-0.000020,0.519976,0.788471,0.122996,0.430674,...,0.474062,0.483298,0.000002,0.516464,0.785509,0.101732,0.431066,0.781687,0.098530,0.0
1,0.429633,0.518555,-0.188891,0.454323,0.495486,0.000037,0.594170,0.760206,0.029811,0.335250,...,0.439054,0.317143,-0.000015,0.663881,0.771235,-0.008332,0.289918,0.694929,0.317332,0.0
2,0.479542,0.452572,-0.579912,0.522247,0.320358,0.000055,0.621352,0.834840,0.080860,0.365188,...,0.566404,0.537541,0.000080,0.603351,0.875083,0.057695,0.530981,0.749985,0.424799,0.0
3,0.502769,0.598824,-0.312952,0.562964,0.701259,0.000136,0.597893,0.916841,-0.091828,0.481447,...,0.629994,0.806218,-0.000025,0.570793,0.894468,0.088400,0.567647,0.868667,0.339632,0.0
4,0.414003,0.573399,-0.001384,0.597498,0.699475,-0.000021,0.566169,0.817170,0.004108,0.565621,...,0.596602,0.631569,0.000035,0.562060,0.805773,-0.040946,0.561076,0.788998,0.512586,0.0


In [3]:
fall_dataset["label"].value_counts()

label
0.0    156
1.0      4
Name: count, dtype: int64

In [4]:
# Tăng số lượng dữ liệu bằng cách nhân bản dữ liệu
for i in range(0, 5):
    fall_dataset = pd.concat([fall_dataset, fall_dataset], ignore_index=True)

In [5]:
fall_dataset["label"].value_counts()

label
0.0    4992
1.0     128
Name: count, dtype: int64

In [6]:
X = fall_dataset.iloc[:, :-1]
y = fall_dataset.iloc[:, -1:]

In [7]:
from imblearn.over_sampling import SMOTE

# Áp dụng Smote để cần bằng dữ liệu bằng cách tạo thêm các mẫu ví dụ cho lớp thiểu số
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [10]:
y.value_counts()

label
0.0      4992
1.0      4992
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

# Tách tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [9]:
# Đếm mỗi loại giá trị của mỗi lớp trong tập huấn luyện và tập kiểm tra
print("Số ví dụ trong tập train: %s" % y_train.count())
print("Số ví dụ trong tập test: %s" % y_test.count())

Số ví dụ trong tập train: label    7987
dtype: int64
Số ví dụ trong tập test: label    1997
dtype: int64


In [11]:
from sklearn.preprocessing import StandardScaler

# Chuẩn hóa các đặc trưng (tùy chọn, nhưng có thể cải thiện hiệu suất cho một số mô hình)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Vẽ biểu đồ 3D

In [12]:
import plotly.graph_objects as go
import pandas as pd

data_sets = [(X_train, y_train, "Train"), (X_test, y_test, "Test")]
train_colors = ["blue", "red"]
test_colors = ["green", "orange"]
visible_dict = {"Normal": True, "Anomaly": True}
fig = go.Figure()

# Tạo biểu đồ 3D cho tập huấn luyện và tập kiểm tra
for (X, y, name_prefix), colors in zip(data_sets, [train_colors, test_colors]):
    balanced_data = pd.DataFrame(
        {"x": X[:, 0], "y": X[:, 1], "z": X[:, 2], "anomaly": y}
    ).sample(n=min(50000, len(X)), random_state=42)

    for anomaly_label, color in zip(range(2), colors):
        scatter = go.Scatter3d(
            x=balanced_data[balanced_data["anomaly"] == anomaly_label]["x"],
            y=balanced_data[balanced_data["anomaly"] == anomaly_label]["y"],
            z=balanced_data[balanced_data["anomaly"] == anomaly_label]["z"],
            mode="markers",
            marker=dict(size=4, color=color, opacity=0.1),
            name=f'{name_prefix} Data: {"Normal" if anomaly_label == 0 else "Anomaly"}',
            visible=visible_dict.get("Normal", True)
            if anomaly_label == 0
            else visible_dict.get("Anomaly", True),
        )
        fig.add_trace(scatter)

# Thêm nút để chuyển đổi giữa các lớp
buttons = [
    dict(
        label="Normal",
        method="update",
        args=[
            {
                "visible": [
                    True,
                    False,
                    True,
                    False,
                    visible_dict["Anomaly"],
                    visible_dict["Anomaly"],
                ]
            }
        ],
    ),
    dict(
        label="Anomaly",
        method="update",
        args=[
            {
                "visible": [
                    False,
                    True,
                    False,
                    True,
                    visible_dict["Normal"],
                    visible_dict["Normal"],
                ]
            }
        ],
    ),
]

# Cập nhật và hiển thị biểu đồ
fig.update_layout(updatemenus=[dict(type="buttons", showactive=False, buttons=buttons)])
fig.show()

# Thuật toán học có giám sát (Supervised Learning Algorithms)

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

supervised_learning_models = {
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
}

In [14]:
from sklearn.model_selection import GridSearchCV

# Huấn luyện mô hình
train_models = {}
for name, classifier in supervised_learning_models.items():
    model = GridSearchCV(classifier, param_grid={})
    model.fit(X_train, y_train)
    train_models[name] = model

In [17]:
# Đánh giá mô hình
for name, model in train_models.items():
    # Tạo dự đoán trên tập kiểm tra X_test
    y_pred = model.predict(X_test)

    # So sánh dự đoán với giá trị từ tập kết quả y_test
    confusion_mat = pd.crosstab(
        y_test, y_pred, rownames=["Actual"], colnames=["Predicted"]
    )

    # Lấy các giá trị TP, TN, FP, FN từ confusion matrix
    TP = confusion_mat.loc[1, 1]
    TN = confusion_mat.loc[0, 0]
    FP = confusion_mat.loc[0, 1]
    FN = confusion_mat.loc[1, 0]

    # Tính toán các chỉ số đánh giá
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = (2 * precision * recall) / (precision + recall)

    # In kết quả
    print(f"{name}:")
    print(confusion_mat)
    print(
        f"Accuracy: {accuracy*100:.2f}%, Precision: {precision*100:.2f}%, Recall: {recall*100:.2f}%, F1: {f1*100:.2f}%"
    )
    print(f"Time: {model.cv_results_['mean_fit_time'][0]:.2f} (s)\n")

KNN:
Predicted   0.0  1.0
Actual              
0.0        1022    0
1.0           0  975
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.03 (s)

Decision Tree:
Predicted   0.0  1.0
Actual              
0.0        1022    0
1.0           0  975
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.16 (s)

Naive Bayes:
Predicted   0.0  1.0
Actual              
0.0        1022    0
1.0           0  975
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.07 (s)



In [18]:
# Lưu mô hình tốt nhất
import pickle

with open("../models/model.pkl", "wb") as file:
    pickle.dump(train_models["KNN"], file)